# M1M3 hardpoint learning
Craig Lage - 14-Apr-23 \
The 17 tons of mirror are supported by 156 pneumatic actuators where 44 are single-axis and provide support only on the axial direction, 100 are dual-axis providing support in the axial and lateral direction, and 12 are dual-axis providing support in the axial and cross lateral directions. \
Positioning is provided by 6 hard points in a hexapod configuration which moves the mirror to a fixed operational position that shall be maintained during telescope operations. The remaining optical elements will be moved relative to this position in order to align the telescope optics. Support and optical figure correction is provided by 112 dual axis and 44 single axis pneumatic actuators. 

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('usdf_efd')
FATABLE = M1M3FATable.FATABLE

In [ ]:
# Times of breakaway test
start = Time("2023-05-03T15:00:00", scale='utc')
end = Time("2023-05-03T17:00:00", scale='utc')

In [ ]:
hardpoints = await client.select_time_series("lsst.sal.MTM1M3.logevent_hardpointTestStatus", "*", start, end)

In [ ]:
len(hardpoints)

In [ ]:
hardpoints.columns

## class HardpointTest(enum.IntEnum): 
    NOTTESTED = 1 
    MOVINGNEGATIVE = 2 
    TESTINGPOSITIVE = 3 
    TESTINGNEGATIVE = 4 
    MOVINGREFERENCE = 5 
    PASSED = 6 
    FAILED = 7 

In [ ]:
for i in range(6):
    column = f"testState{i}"
    hardpoints[column].plot()

## First look at the forces and displacements during the hardpoint breakaway test

In [ ]:
nx = 3; ny = 2
fig, axs = plt.subplots(nx, ny, figsize=(8,10))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
timestamp = hardpoints.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
plt.suptitle(f"Hardpoints tests {timestamp}")
for i in range(nx):
    for j in range(ny):
        ax = axs[i,j]
        index = i * ny + j
        plotStart = hardpoints[hardpoints[f'testState{index}']==2]['private_kafkaStamp'][0] - 1.0
        plotEnd = hardpoints[hardpoints[f'testState{index}']==6]['private_kafkaStamp'][0] + 1.0
        testingPositive = hardpoints[hardpoints[f'testState{index}']==3]['private_kafkaStamp'][0]
        testingPositive = Time(testingPositive, format='unix_tai', scale='tai').utc.isot
        testingNegative = hardpoints[hardpoints[f'testState{index}']==4]['private_kafkaStamp'][0]
        testingNegative = Time(testingNegative, format='unix_tai', scale='tai').utc.isot
        movRef = hardpoints[hardpoints[f'testState{index}']==5]['private_kafkaStamp'][0]
        movRef = Time(movRef, format='unix_tai', scale='tai').utc.isot
        start = Time(plotStart, format='unix_tai', scale='tai')
        end = Time(plotEnd, format='unix_tai', scale='tai')
        hardpointData = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", "*", start.utc, end.utc)
        hardpointData[f'displacement{index}'].plot(ax=ax, color='green', label='Displacement')
        ax.set_ylim(-0.01,0.025)
        ax.axvline(testingPositive, ls='--')
        ax.axvline(testingNegative, ls='--')
        ax.axvline(movRef, ls='--')
        ax2 = ax.twinx()
        hardpointData[f'measuredForce{index}'].plot(ax=ax2, color='red', label='Force')
        ax.set_title(f'Hardpoint {index}')
        ax.set_ylabel('Displacement (m?)')
        ax.legend(loc='upper left', fontsize=6)
        ax2.legend(loc='upper right', fontsize=6)
        ax2.set_ylim(-4000, 4000)
        ax2.set_ylabel('Force (nt)')
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Hardpoint_Test{timestamp}.pdf")

## Look at the hardpoint displacements

In [ ]:
plt.subplot(2,1,1)
hardpointData['xPosition'].plot(label='X')
hardpointData['yPosition'].plot(label='Y')
hardpointData['zPosition'].plot(label='Z')
plt.legend()
plt.subplot(2,1,2)
hardpointData['xRotation'].plot(label='XRot')
hardpointData['yRotation'].plot(label='YRot')
hardpointData['zRotation'].plot(label='ZRot')
plt.legend()

## Now blow up the region where the force changes rapidly

In [ ]:
states = [['Pos',3,4], ['Neg',4,5]]
nx = 3; ny = 2
tPlot = 15.0
fig, axs = plt.subplots(nx, ny, figsize=(8,10))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
timestamp = hardpoints.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
plt.suptitle(f"Hardpoints tests {timestamp}")
for i in range(nx):
    for j in range(ny):
        ax = axs[i,j]
        index = i * ny + j
        for [name,startState, endState] in states:
            getStart = hardpoints[hardpoints[f'testState{index}']==startState]['private_kafkaStamp'][0]
            getEnd = hardpoints[hardpoints[f'testState{index}']==endState]['private_kafkaStamp'][0]
            start = Time(getStart, format='unix_tai', scale='tai')
            end = Time(getEnd, format='unix_tai', scale='tai')
            hardpointData = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", \
                            [f'measuredForce{index}', 'timestamp'], start.utc, end.utc)
            plotStart = hardpointData.loc[abs(hardpointData[f'measuredForce{index}']) < 100.0]['timestamp'][-1] - tPlot/2.0
            plotEnd = plotStart + tPlot
            start = Time(plotStart, format='unix_tai', scale='tai')
            end = Time(plotEnd, format='unix_tai', scale='tai')
            hardpointData = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", \
                            [f'displacement{index}', f'measuredForce{index}'], start.utc, end.utc)
            force = hardpointData[f'measuredForce{index}']
            disp = hardpointData[f'displacement{index}']
            ax.plot(disp, force, label=name)
            ax.set_title(f'Hardpoint {index}')
            ax.set_ylabel('Force (nt)')
            ax.set_ylim(-4000, 4000)
            ax.set_xlabel('Displacement(m)')
        ax.legend(loc='upper left')
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Hardpoint_Test_Blowup_{timestamp}.pdf")

## Now look at the hardpoint forces during the force balance test

In [ ]:
# Times of force balance test
start = Time("2023-04-18 16:10:00Z", scale='utc')
end = Time("2023-04-18 16:15:00Z", scale='utc')

In [ ]:
hardpoint_data = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", "*", start, end)

In [ ]:
plots = [f'measuredForce{i}' for i in range(6)]
fig, axs = plt.subplots(3,2,figsize=(8,10))
plt.subplots_adjust(hspace=0.6, wspace=0.3)
plot_counter = 0
for i in range(3):
    for j in range(2):
        smoothed_hardpoint_data = hardpoint_data[plots[plot_counter]].rolling(100).mean()
        axs[i][j].set_title(plots[plot_counter])
        axs[i][j].set_ylabel(f'Hardpoint force (nt)')
        axs[i][j].set_ylim(-100,100)
        smoothed_hardpoint_data.plot(ax=axs[i][j], color='blue')
        plot_counter += 1
timestamp = hardpoint_data.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Hardpoint_Forces_{timestamp}.png")